In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import pandas as pd
from pyspark.sql import SQLContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
spark = SparkSession(sc)

In [2]:
from numpy import array
import sys
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree
from pyspark.mllib.linalg import SparseVector

## Classification

### Decision trees

#### Creating an RDD of LabeledPoint Objects

Để xây dựng một ví dụ về bộ phân loại cây quyết định bằng cách sử dụng tập dữ liệu huấn luyện,trước tiên bạn cần phải tạo một RDD gồm LabeledPoint (pyspark.mllib.regression.LabeledPoint) các đối tượng. MỘTĐối tượng LabeledPoint chứa thuộc tính nhãn hoặc lớp cho một trường hợp,cùng với các thuộc tính cá thể liên quan.

Thuộc tính đối tượng LabeledPoint phải là giá trị float hoặc các đối tượng có thểđược chuyển đổi thành các giá trị float, chẳng hạn như Boolean hoặc int.

In [3]:
from pyspark.mllib.regression import LabeledPoint
outlook = {"sunny": 0.0, "overcast": 1.0, "rainy": 2.0}
labeledpoints = [
 LabeledPoint(0.0,[outlook["sunny"],85,85,False]),
 LabeledPoint(0.0,[outlook["sunny"],80,90,True]),
 LabeledPoint(1.0,[outlook["overcast"],83,86,False]),
 LabeledPoint(1.0,[outlook["rainy"],70,96,False]),
 LabeledPoint(1.0,[outlook["rainy"],68,80,False]),
 LabeledPoint(1.0,[outlook["rainy"],68,80,False]),
 LabeledPoint(1.0,[outlook["rainy"],68,80,False]),
 LabeledPoint(0.0,[outlook["rainy"],65,70,True]),
 LabeledPoint(1.0,[outlook["overcast"],64,65,True]),
 LabeledPoint(0.0,[outlook["sunny"],72,95,False]),
 LabeledPoint(1.0,[outlook["sunny"],69,70,False]),
 LabeledPoint(1.0,[outlook["sunny"],75,80,False]),
 LabeledPoint(1.0,[outlook["sunny"],75,70,True]),
 LabeledPoint(1.0,[outlook["overcast"],72,90,True]),
 LabeledPoint(1.0,[outlook["overcast"],81,75,False]),
 LabeledPoint(0.0,[outlook["rainy"],71,91,True])
 ]
data = sc.parallelize(labeledpoints)

In [4]:
data.collect()

[LabeledPoint(0.0, [0.0,85.0,85.0,0.0]),
 LabeledPoint(0.0, [0.0,80.0,90.0,1.0]),
 LabeledPoint(1.0, [1.0,83.0,86.0,0.0]),
 LabeledPoint(1.0, [2.0,70.0,96.0,0.0]),
 LabeledPoint(1.0, [2.0,68.0,80.0,0.0]),
 LabeledPoint(1.0, [2.0,68.0,80.0,0.0]),
 LabeledPoint(1.0, [2.0,68.0,80.0,0.0]),
 LabeledPoint(0.0, [2.0,65.0,70.0,1.0]),
 LabeledPoint(1.0, [1.0,64.0,65.0,1.0]),
 LabeledPoint(0.0, [0.0,72.0,95.0,0.0]),
 LabeledPoint(1.0, [0.0,69.0,70.0,0.0]),
 LabeledPoint(1.0, [0.0,75.0,80.0,0.0]),
 LabeledPoint(1.0, [0.0,75.0,70.0,1.0]),
 LabeledPoint(1.0, [1.0,72.0,90.0,1.0]),
 LabeledPoint(1.0, [1.0,81.0,75.0,0.0]),
 LabeledPoint(0.0, [2.0,71.0,91.0,1.0])]

#### Training a Decision Tree Model with Spark MLlib
Sử dụng RDD chứa các đối tượng LabeledPoint được tạo, bạn bây giờ có thể đào tạo mô hình cây quyết định bằng cách sử dụng Hàm DecisionTree.trainClassifier() trong mllib Spark

Hàm DecisionTree.trainClassifier () tạo một mô hình bằng cáchđào tạo dữ liệu, một bộ sưu tập song song các đối tượng LabeledPoint. Các đối số numClasses chỉ định có bao nhiêu lớp rời rạc để dự đoán; trong nàytrường hợp, nó là hai vì ví dụ chỉ đơn giản là dự đoán một kết quả nhị phân là có / không.Đối số categoricalFeaturesInfo là một từ điển hoặc bản đồ chỉ định đối tượng địa lý nào là phân loại và có bao nhiêu giá trị phân loại mỗi những tính năng đó có thể thực hiện. Trong trường hợp này, bạn cần hướng phương thức trainClassifier () mà các giá trị đại diện cho triển vọng danh mục rời rạc — ví dụ: "nắng" hoặc "mưa" hoặc "u ám".Bất kỳ tính năng nào không được chỉ định trong đối số categoricalFeaturesInfo được coi là liên tục.

In [5]:
from pyspark.mllib.tree import DecisionTree
#categoricalFeaturesInfo = {}
#model = DecisionTree.trainClassifier(data=data, numClasses=2, categoricalFeaturesInfo=categoricalFeaturesInfo)
model = DecisionTree.trainClassifier(data=data,
 numClasses=2,
 categoricalFeaturesInfo={0: 3})
print(model.toDebugString())

DecisionTreeModel classifier of depth 4 with 13 nodes
  If (feature 2 <= 82.5)
   If (feature 1 <= 66.5)
    If (feature 0 in {2.0})
     Predict: 0.0
    Else (feature 0 not in {2.0})
     Predict: 1.0
   Else (feature 1 > 66.5)
    Predict: 1.0
  Else (feature 2 > 82.5)
   If (feature 0 in {0.0})
    Predict: 0.0
   Else (feature 0 not in {0.0})
    If (feature 0 in {2.0})
     If (feature 1 <= 70.5)
      Predict: 1.0
     Else (feature 1 > 70.5)
      Predict: 0.0
    Else (feature 0 not in {2.0})
     Predict: 1.0



#### Using a Spark MLlib Decision Tree Model to Classify New Data
Dầu vào outlook = "overcast",nhiệt độ = 85, độ ẩm = 85 và gió = có, quyết định chơi là 1,0 --> đi chơi golf. Điều này tuân theo logic từ cây quyết định bạn đã tạo.

In [6]:
model.predict([outlook["overcast"],85,85,True])

1.0

### Naive Bayes

#### Implementing a Naive Bayes Classifier Using Spark MLlib

In [7]:
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
model = NaiveBayes.train(data=data, lambda_=1.0)
model.predict([1.0,85,85,True])

1.0

In [8]:
sc.stop()

## Collaborative Filtering Using Spark MLlib

### Exercise: Implementing a Recommender Using Spark MLlib
Exercise: Implementing a Recommender Using Spark MLlib

Tập dữ liệu chứa 100.000 xếp hạng bởi 943 người dùng trên 1.682 mục, với mỗi người dùng đã xếp hạng ít nhất 20 phim.

user id | item id | rating | timestamp

In [9]:
#Import the required MLlib libraries:
import sys
from pyspark import SparkConf, SparkContext
from pyspark.mllib.recommendation \
import ALS, MatrixFactorizationModel, Rating
conf = SparkConf().setAppName('Movielens Recommender')
sc = SparkContext(conf=conf)

In [10]:
#Load the Movielens dataset and create an RDD containing Rating objects:
data = sc.textFile('data/movielens.csv')
ratings = data.map(lambda x: x.split(','))
ratings.first()

['196', '242', '3', '881250949']

In [17]:
data = sc.textFile('data/movielens.dat')
ratings = data.map(lambda x: x.split('\t')) \
 .map(lambda x: Rating(int(x[0]), int(x[1]), float(x[2]))) #Bỏ qua timestamp
ratings.first()

Rating(user=196, product=242, rating=3.0)

In [18]:
#Train a model using the ALS algorithm:
rank = 10
numIterations = 10
model = ALS.train(ratings, rank, numIterations)

Lưu ý rằng rank và numIterations là các tham số điều chỉnh thuật toán;Rank là số lượng các yếu tố tiềm ẩn trong mô hình và numIterations là số lần lặp lại để chạy

Bây giờ có thể kiểm tra mô hình dựa trên cùng một tập dữ liệu mà không cần rating(sử dụng mô hình để dự đoán thuộc tính này). Sau đó so sánh kết quả của dự đoán với xếp hạng thực tế để xác định sai số bình phương trung bình,đo độ chính xác của mô hình:

In [29]:
testdata = ratings.map(lambda p: (p[0], p[1]))
testdata.take(5)

[(196, 242), (186, 302), (22, 377), (244, 51), (166, 346)]

In [30]:
predictions = model.predictAll(testdata) \
 .map(lambda r: ((r[0], r[1]), r[2]))
predictions.take(5)

[((316, 1084), 3.680324435930144),
 ((504, 1084), 4.3252875839772855),
 ((424, 1084), 5.020947172396465),
 ((541, 1084), 4.733278984846265),
 ((181, 1084), 1.8892183936344953)]

In [33]:
ratesAndPreds = ratings.map(lambda r: ((r[0], r[1]), r[2])) \
 .join(predictions)
ratesAndPreds.take(5)

[((186, 302), (3.0, 3.3186171078919497)),
 ((115, 265), (2.0, 3.0859953266379687)),
 ((253, 465), (5.0, 4.270124091340714)),
 ((50, 246), (3.0, 3.3556865350317544)),
 ((225, 193), (4.0, 4.439608183164759))]

In [39]:
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2) \
 .mean()
print("Mean Squared Error = " + str(MSE))

Mean Squared Error = 0.48192716404188984


In [40]:
#Để lưu mô hình để sử dụng với các đề xuất mới, hãy sử dụnghàm model.save()
model.save(sc, "ratings_model")

In [41]:
#Triển khai mô hình dựa trên dữ liệu thời gian thực từ Spark DStream — sử dụng Hàm MatrixFactorizationModel.load()
from pyspark.mllib.recommendation import MatrixFactorizationModel
reloaded_model = MatrixFactorizationModel.load(sc, "ratings_model")

In [42]:
reloaded_model

In [1]:
sc.stop()

## Clustering Using Spark MLlib

### Training a k-Means Clustering Model Using Spark MLlib

Để triển khai k-means trong Spark, hãy sử dụng gói pyspark.mllib.clustering.KMeans.Trình bày cách đào tạo mô hình học máy phân cụm k-mean bằng cách sử dụng tập dữ liệu kmeans_data mẫu được cung cấp như một phần của bản phát hành Spark.

In [8]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array
from math import sqrt
from pyspark import SparkConf, SparkContext
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [11]:
# Load and parse the data
data = sc.textFile("data/mllib/kmeans_data.txt")
data.collect()

['0.0 0.0 0.0',
 '0.1 0.1 0.1',
 '0.2 0.2 0.2',
 '9.0 9.0 9.0',
 '9.1 9.1 9.1',
 '9.2 9.2 9.2']

In [12]:
parsedData = data.map(lambda line: array( \
 [float(x) for x in line.split(' ')]))
parsedData.collect()

[array([0., 0., 0.]),
 array([0.1, 0.1, 0.1]),
 array([0.2, 0.2, 0.2]),
 array([9., 9., 9.]),
 array([9.1, 9.1, 9.1]),
 array([9.2, 9.2, 9.2])]

In [13]:
# Build the model (cluster the data)
clusters = KMeans.train(parsedData, 2, maxIterations=10,initializationMode="random")
clusters

### Evaluating a k-Means Clustering Model

In [14]:
# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
 center = clusters.centers[clusters.predict(point)]
 return sqrt(sum([x**2 for x in (point - center)]))

In [15]:
WSSSE = parsedData.map(lambda point: error(point))
WSSSE.collect()

[0.17320508075688776,
 0.0,
 0.17320508075688776,
 0.17320508075688404,
 3.076740298213702e-15,
 0.1732050807568902]

In [19]:
WSSSE = parsedData.map(lambda point: error(point)) \
 .reduce(lambda x, y: x + y)

In [18]:
print("Within Set Sum of Squared Error = " + str(WSSSE))
# returns:
# Within Set Sum of Squared Error = 0.692820323028

Within Set Sum of Squared Error = 0.6928203230275529


### Saving and Reloading a Clustering Model

In [20]:
# Save and load model
clusters.save(sc, "kmeans_model")
reloaded_model = KMeansModel.load(sc, "kmeans_model")